### Task 5: Function Calling

In [18]:
import os
from openai import OpenAI

client = OpenAI(
    api_key='lm-studio',  
    base_url="http://localhost:1234/v1"
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say only 'This is a test!' ",
        }
    ],
    model="llama-3.2-1b-instruct",
)
print(chat_completion.choices[0].message.content)

This is a test!


In [44]:

import json

from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

# A dummy function that always returns the same weather information
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    print(json.dumps(weather_info))


def run_conversation():
    messages = [
        {"role": "system",
          "content": 
          """You are a helpful assistant that uses function calls based on user input and call function "get_current_weather". Only use the functions you have been provided with. Using this 
           provided function definition generates the Output only in the Json format. 
           Please provide the response strictly in valid JSON format and no other extra strings or punctuations, without any additional text or explanation. 

           
          The function definition for extracting the information from given user prompt: 

         {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city San Francisco",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
         }

           Ensure the JSON is well-formed and adheres to the following structure:
           Assistant: {
            "name": "get_current_weather",
            "arguments": {"location": "Berlin", "unit": "celsius"}
           }
           
           """},
        {"role": "user", "content": "What's the weather like in Boston?"}]

    response = client.chat.completions.create(
        model="llama-3.2-1b-instruct",
        messages=messages,
        max_tokens=50,
    
    )

    
    response_message = response.choices[0].message.content


    if "Assistant" in response_message: 
       
       print("Response Message: ", response_message)
       print("Type of response message:", type(response_message))

       data = json.loads(str(response_message))

       # Extract values
       name = data["Assistant"]["name"]
       location = data["Assistant"]["arguments"]["location"]
       unit = data["Assistant"]["arguments"]["unit"]

       print(f"Name: {name}")
       print(f"Location: {location}")
       print(f"Unit: {unit}")
       
       if name == "get_current_weather": 
        
        if location != None : 
            if unit != None: 
              
                get_current_weather(location, unit)
            else: 
               
               get_current_weather(location)
        else:
           return "No location passed"
       else: 
          return "No function called"



run_conversation()

Response Message:  {
  "Assistant": {
    "name": "get_current_weather",
    "arguments": {"location": "Boston", "unit": "fahrenheit"}
  }
}
Type of response message: <class 'str'>
Name: get_current_weather
Location: Boston
Unit: fahrenheit
{"location": "Boston", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}
